# 데이터 다운로드

# 주의 사항 : 5시간 걸립니다. ( svm에 데이터가 너무 커서)

In [31]:
import pandas as pd
mnist_test = pd.read_csv('emnist-digits-test.csv',header=None)
mnist = pd.read_csv('emnist-digits-train.csv',header=None) 
#엑셀 다운로드

In [32]:
import numpy as np
mnist = np.array(mnist)
mnist_test = np.array(mnist_test)

In [33]:
mnist_train_target = mnist[:,0]
mnist_train_data = mnist[:,1:]
mnist_test_target = mnist_test[:,0]
mnist_test_data = mnist_test[:,1:]
#타켓 레이블과 특성값으로 나눔

In [34]:
import numpy as np

shuffle_index = np.random.permutation(240000)
mnist_train_data, mnist_train_target = mnist_train_data[shuffle_index], mnist_train_target[shuffle_index]

# Feature Scaling

### StandardScaler

In [35]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
mnist_train_data_scale = scaler.fit_transform(mnist_train_data.astype(np.float32))
mnist_test_data_scale  = scaler.transform(mnist_test_data.astype(np.float32))

* Min-Max스케일링은0-15사이의모든값을0-0.15로만들어버림
* 표준화는이상치를포함하여데이터분포를1로만드므로영향을적게받음

# Feature Engineering

In [36]:
#mnist_train_data = np.mean(mnist_train_data, axis = 1)
#mnist_train_data=mnist_train_data.reshape(-1,1)

* 픽셀 평균: 특성이 줄어들어서 결과 안좋아짐

# TrainMode

## SGD

In [37]:
'''from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=5, tol=-np.infty, random_state=42)
sgd_clf.fit(mnist_train_data_scale, mnist_train_target)
#SGD분류기 max_iter 통과하는 데이터 수 온라인 학습'''

'from sklearn.linear_model import SGDClassifier\n\nsgd_clf = SGDClassifier(max_iter=5, tol=-np.infty, random_state=42)\nsgd_clf.fit(mnist_train_data_scale, mnist_train_target)\n#SGD분류기 max_iter 통과하는 데이터 수 온라인 학습'

## RandomForest

In [38]:
'''from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=100, max_depth=20, n_jobs=-1, random_state=42)
forest_clf.fit(mnist_train_data_scale, mnist_train_target)'''

'from sklearn.ensemble import RandomForestClassifier\nforest_clf = RandomForestClassifier(n_estimators=100, max_depth=20, n_jobs=-1, random_state=42)\nforest_clf.fit(mnist_train_data_scale, mnist_train_target)'

* n_estimators: 랜덤 포레스트 안의 결정 트리 갯수
* n_estimators 10: 0.97
* n_estimators 100: 0.981175
* max_features: n_features로 설정하면 트리의 각 분기에서 모든 특성을 고려하므로 특성 선택에 무작위성이 들어가지 않습니다. 그래서 default로 적게 

## DecisionTree

In [39]:
'''from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(max_depth=20, random_state=42)
tree_clf.fit(mnist_train_data_scale, mnist_train_target)'''

'from sklearn.tree import DecisionTreeClassifier\ntree_clf = DecisionTreeClassifier(max_depth=20, random_state=42)\ntree_clf.fit(mnist_train_data_scale, mnist_train_target)'

* default : 0.92
* max_depth 15: 0.9273
* max_depth 20:0.9274
* max_depth 25:0.92505 

## SVM

**1. LinearSVC**

In [40]:
'''from sklearn.svm import LinearSVC
svm_clf= LinearSVC(max_iter=10,loss="hinge", random_state=42)
svm_clf.fit(mnist_train_data_scale, mnist_train_target)'''

'from sklearn.svm import LinearSVC\nsvm_clf= LinearSVC(max_iter=10,loss="hinge", random_state=42)\nsvm_clf.fit(mnist_train_data_scale, mnist_train_target)'

* C: 마진 오류 조절 ( c크면 오류 작아짐)
* tol(허용오차): 0이면 정상적 분류 , 0 < tol < 1 , 1이상 잘못 분류
* 0.87

**2. 다항특성과 LinearSVC**

In [42]:
'''import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.preprocessing import PolynomialFeatures

scaler = StandardScaler()
mnist_train_data_scale = scaler.fit_transform(mnist_train_data)
poly = PolynomialFeatures(3)
y = poly.fit_transform(mnist_train_data_scale, y=None)
polynomial_svm= LinearSVC(max_iter=10,loss="hinge", random_state=42)
polynomial_svm.fit(y, mnist_train_target)'''


'import numpy as np\nfrom sklearn import datasets\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.svm import LinearSVC\nfrom sklearn.preprocessing import PolynomialFeatures\n\nscaler = StandardScaler()\nmnist_train_data_scale = scaler.fit_transform(mnist_train_data)\npoly = PolynomialFeatures(3)\ny = poly.fit_transform(mnist_train_data_scale, y=None)\npolynomial_svm= LinearSVC(max_iter=10,loss="hinge", random_state=42)\npolynomial_svm.fit(mnist_train_data_scale, mnist_train_target)'

**3. SVC**

In [12]:
'''from sklearn.svm import SVC
poly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C= 100, gamma='auto', max_iter=2000)
poly_kernel_svm_clf.fit(mnist_train_data_scale, mnist_train_target)'''

'from sklearn.svm import SVC\npoly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C= 100, gamma=\'auto\', max_iter=2000)\npoly_kernel_svm_clf.fit(mnist_train_data_scale, mnist_train_target)'

* coef0 - 고차항 영향
* coef0 1: 044 (max_iter = 10 , 빠른 확인)
* coef0 100:0.38 (max_iter = 10 , 빠른 확인)
* C 5: 0.4 
* C 20:0.46
* 오차 0.988 (max_iter = -1)
* max_iter = 2000 0.987

**4. RBF SVC**

In [13]:
'''from sklearn.svm import SVC
rbf_kernel_svm_clf = SVC(degree=3, kernel="rbf", gamma=0.005, C=5, max_iter = 500)
rbf_kernel_svm_clf.fit(mnist_train_data_scale, mnist_train_target)  
# 하루돌리고 오류 포기'''

'from sklearn.svm import SVC\nrbf_kernel_svm_clf = SVC(degree=3, kernel="rbf", gamma=0.005, C=5, max_iter = 500)\nrbf_kernel_svm_clf.fit(mnist_train_data_scale, mnist_train_target)  \n# 하루돌리고 오류 포기'

* 0,975 (max_iter = 100)

## Logistic Regression

In [14]:
'''from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs',C=50 ,random_state=42)
log_reg.fit(mnist_train_data_scale, mnist_train_target)'''

"from sklearn.linear_model import LogisticRegression\nlog_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs',C=50 ,random_state=42)\nlog_reg.fit(mnist_train_data_scale, mnist_train_target)"

## Voting classifiers

In [15]:
'''from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

sgd_clf = Classifier(max_iter=5, tol=-np.infty, random_state=42)
forest_clf = RandomForestClassifier(n_estimators=100, max_depth=20, n_jobs=-1, random_state=42)
poly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C= 100, gamma='auto', max_iter=2000)'''

'from sklearn.ensemble import VotingClassifier\nfrom sklearn.svm import SVC\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.linear_model import SGDClassifier\n\nsgd_clf = Classifier(max_iter=5, tol=-np.infty, random_state=42)\nforest_clf = RandomForestClassifier(n_estimators=100, max_depth=20, n_jobs=-1, random_state=42)\npoly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C= 100, gamma=\'auto\', max_iter=2000)'

In [16]:
'''voting_clf = VotingClassifier(
    estimators=[('fc', forest_clf), ('sc', sgd_clf), ('svc', poly_kernel_svm_clf)],
    voting='hard')'''

"voting_clf = VotingClassifier(\n    estimators=[('fc', forest_clf), ('sc', sgd_clf), ('svc', poly_kernel_svm_clf)],\n    voting='hard')"

In [17]:
'''voting_clf.fit(mnist_train_data_scale, mnist_train_target)'''

'voting_clf.fit(mnist_train_data_scale, mnist_train_target)'

In [18]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=100, max_depth=20, n_jobs=-1, random_state=42)
poly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C= 100, gamma='auto', max_iter=2500, probability= True)

In [19]:
voting_clf = VotingClassifier(
    estimators=[('fc', forest_clf), ('svc', poly_kernel_svm_clf)],
    voting='soft')

In [20]:
voting_clf.fit(mnist_train_data_scale, mnist_train_target)

C:\Users\Ubicomp_Lab\Anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=2500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


VotingClassifier(estimators=[('fc',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=20,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=100,
                                                     n_jobs=-1, oob_score=Fals

* 0.99

# Performance Measure

## Cross-validation

**1. SGD**

In [21]:
#from sklearn.model_selection import cross_val_score
#cross_val_score(sgd_clf, mnist_train_data_scale,mnist_train_target, cv=10, scoring="accuracy")
#SGD분류기 10-fold 교차검증

**2. RandomForest**

In [22]:
#cross_val_score(forest_clf, mnist_train_data_scale,mnist_train_target, cv=10, scoring="accuracy")
#randomforest 분류기 10-fold 교차검증

**3. DecisionTree**

In [23]:
#cross_val_score(tree_clf, mnist_train_data_scale,mnist_train_target, cv=10, scoring="accuracy")
#DT분류기 10-fold 교차검증

**4. LinearSVC**

In [24]:
#cross_val_score(svm_clf, mnist_train_data_scale,mnist_train_target, cv=10, scoring="accuracy")

**4. 다항특성과 LinearSVC(안됨)**

In [25]:
#cross_val_score(polynomial_svm_clf, mnist_train_data_scale ,mnist_train_target, cv=10, scoring="accuracy")
# 왜인지는 모르겠고 안된다.

**5. SVC**

In [26]:
#cross_val_score(poly_kernel_svm_clf, mnist_train_data_scale,mnist_train_target, cv=10, scoring="accuracy")

**6. RBF SVC**

In [27]:
#cross_val_score(rbf_kernel_svm_clf, mnist_train_data_scale,mnist_train_target, cv=10, scoring=  "accuracy")

**6. Voting classifiers**

In [43]:
#from sklearn.model_selection import cross_val_score
#cross_val_score(voting_clf, mnist_train_data_scale,mnist_train_target, cv=3, scoring=  "accuracy")

## 오차행렬

In [44]:
#conf_mx = confusion_matrix(mnist_train_target, y_train_pred)
#conf_mx

# 정확도

In [30]:
from sklearn.metrics import accuracy_score
y_pred = voting_clf.predict(mnist_test_data_scale)
accuracy_score(mnist_test_target, y_pred)

0.990125

* 0.99